In [ ]:
!pip install augly[all]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)


In [ ]:
import os
os.chdir('/content/drive/MyDrive/GT/DL/project/meme-challenge')
!pwd

In [ ]:
from PIL import Image

In [ ]:
import pandas as pd

data_dir = './hateful_memes/'
#dev_path = data_dir + "dev_unseen.jsonl"
test_path = data_dir + "adv_test_unseen.jsonl"

data = pd.read_json(path_or_buf=test_path, lines=True)

## 1: Emoji Sticker Augmentation

Use emoji dataset from https://www.kaggle.com/datasets/subinium/emojiimage-dataset

If hateful (label 1), add random emoji from list of positive emojis, randomly somewhere in image.

If not hateful, add random emoji from list of negative emojis.

In [ ]:
import augly.image as imaugs
import augly.utils as utils
import augly.text as textaugs
import random
from tqdm import tqdm

def get_emoji_list():
    positive_emojis = []
    negative_emojis = []

    for img in os.listdir("./emojis/positive/"):
        positive_emojis.append(img)

    for neg_img in os.listdir("./emojis/negative/"):
        negative_emojis.append(neg_img)
    return positive_emojis, negative_emojis

def apply_emoji_aug(row):

    positve_emojis, negative_emojis = get_emoji_list()

    image = Image.open(f"./hateful_memes/{row['img']}")

    print(row['label'])

    if row['label'] == 1:
        # hateful, add positive emoji
        emoji_path = "./emojis/positive/"+positive_emojis[random.randint(0, len(positive_emojis) - 1)]
    else:
        # not hateful, add negative emoji
        emoji_path = "./emojis/negative/"+negative_emojis[random.randint(0, len(negative_emojis) - 1)]

    aug = imaugs.Compose(
            [
                imaugs.OverlayEmoji(emoji_path=emoji_path, emoji_size=0.1, x_pos=random.uniform(0.3,0.7), y_pos=random.uniform(0.3,0.7), opacity=1.0),
            ]
        )
    aug_img = aug(image)
    aug_img.show()
    img_save_path = f"aug/emoji_aug_{row['img'].replace(r'img/', '')}"
    aug_img.save(img_save_path)
    row['aug_img'] = img_save_path

    return row

In [ ]:
tqdm.pandas()
emoji_aug_data = data.progress_apply(apply_emoji_aug,axis=1)

In [ ]:
import json
import pandas as pd

with open("emoji_aug_dev_unseen.jsonl", "w") as f:
    f.write(emoji_aug_data.to_json(orient='records', lines=True, force_ascii=False).replace('\/', '/'))

In [ ]:
tqdm.pandas()
emoji_aug_data_test = data.progress_apply(apply_emoji_aug,axis=1)

In [ ]:
import json
import pandas as pd

with open("emoji_aug_test_unseen.jsonl", "w") as f:
    f.write(emoji_aug_data_test.to_json(orient='records', lines=True, force_ascii=False).replace('\/', '/'))

## Image Augmentation 2: Grayscale

In [ ]:
def apply_grayscale_aug(row):

    image = Image.open(f"./hateful_memes/{row['img']}")

    aug = imaugs.Compose([imaugs.Grayscale(mode='luminosity', p=1.0)])
    aug_img = aug(image)
    #aug_img.show()
    img_save_path = f"aug/grayscale_aug_{row['img'].replace(r'img/', '')}"
    aug_img.save(img_save_path)
    row['aug_img'] = img_save_path
    return row

In [ ]:
grayscale_aug_data_test = data.progress_apply(apply_grayscale_aug,axis=1)

In [ ]:
grayscale_aug_data

,id,img,label,text,aug_img
0,76432,img/76432.png,0,you thinking what i'm thinking?,aug/grayscale_aug_76432.png
1,14270,img/14270.png,0,a brilliant mind,aug/grayscale_aug_14270.png
2,56947,img/56947.png,0,"pro gamer 6.000.000 kills, 1 death",aug/grayscale_aug_56947.png
3,35174,img/35174.png,0,lets end poaching,aug/grayscale_aug_35174.png
4,39264,img/39264.png,0,my wife called me a good husband thats a huge ...,aug/grayscale_aug_39264.png
...,...,...,...,...,...
535,83675,img/83675.png,0,i'm gonna be like phelps one day,aug/grayscale_aug_83675.png
536,37198,img/37198.png,0,when you're so relaxed you can feel yourself g...,aug/grayscale_aug_37198.png
537,48670,img/48670.png,0,look at this sandwich maker club i found on wi...,aug/grayscale_aug_48670.png
538,9863,img/09863.png,0,diverse group of women,aug/grayscale_aug_09863.png


In [ ]:
with open("grayscale_aug_dev_unseen.jsonl", "w") as f:
    f.write(grayscale_aug_data.to_json(orient='records', lines=True, force_ascii=False).replace('\/', '/'))

In [ ]:
with open("grayscale_aug_test_unseen.jsonl", "w") as f:
    f.write(grayscale_aug_data_test.to_json(orient='records', lines=True, force_ascii=False).replace('\/', '/'))

## Image Augmentation 3: Random Noise

In [ ]:
def apply_random_noise_aug(row):

    image = Image.open(f"./hateful_memes/{row['img']}")

    aug = imaugs.Compose([imaugs.RandomNoise(mean=0.0,var=0.005)])
    aug_img = aug(image)
    #aug_img.show()
    img_save_path = f"aug/noise_aug_{row['img'].replace(r'img/', '')}"
    aug_img.save(img_save_path)
    row['aug_img'] = img_save_path
    return row

tqdm.pandas()
aug_noise_data_test = data.apply(apply_random_noise_aug,axis=1)

In [ ]:
aug_noise_data

,id,img,label,text,aug_img
0,76432,img/76432.png,0,you thinking what i'm thinking?,aug/noise_aug_76432.png
1,14270,img/14270.png,0,a brilliant mind,aug/noise_aug_14270.png
2,56947,img/56947.png,0,"pro gamer 6.000.000 kills, 1 death",aug/noise_aug_56947.png
3,35174,img/35174.png,0,lets end poaching,aug/noise_aug_35174.png
4,39264,img/39264.png,0,my wife called me a good husband thats a huge ...,aug/noise_aug_39264.png
...,...,...,...,...,...
535,83675,img/83675.png,0,i'm gonna be like phelps one day,aug/noise_aug_83675.png
536,37198,img/37198.png,0,when you're so relaxed you can feel yourself g...,aug/noise_aug_37198.png
537,48670,img/48670.png,0,look at this sandwich maker club i found on wi...,aug/noise_aug_48670.png
538,9863,img/09863.png,0,diverse group of women,aug/noise_aug_09863.png


In [ ]:
with open("added_noise_aug_dev_unseen.jsonl", "w") as f:
    f.write(aug_noise_data.to_json(orient='records', lines=True, force_ascii=False).replace('\/', '/'))

In [ ]:
with open("added_noise_aug_text_unseen.jsonl", "w") as f:
    f.write(aug_noise_data_test.to_json(orient='records', lines=True, force_ascii=False).replace('\/', '/'))

## Text Augmentation

In [ ]:
import augly.text as txtaugs

data
data['augmented_text']= txtaugs.replace_similar_unicode_chars(list(data['text']))

In [ ]:
data

In [ ]:
with open("similar_unicode_chars_test_unseen.jsonl", "w") as f:
    f.write(data.to_json(orient='records', lines=True, force_ascii=False).replace('\/', '/'))

In [ ]:
data['augmented_text']= txtaugs.swap_gendered_words(list(data['text']))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open("swap_gendered_words_test_unseen.jsonl", "w") as f:
    f.write(data.to_json(orient='records', lines=True, force_ascii=False).replace('\/', '/'))

In [ ]:
data['augmented_text']= txtaugs.simulate_typos(list(data['text']))

In [ ]:
with open("simulate_typos_test_unseen.jsonl", "w") as f:
    f.write(data.to_json(orient='records', lines=True, force_ascii=False).replace('\/', '/'))